## Synthetic data

In [1]:
%load_ext autoreload 
%autoreload 2

import sys
import ast

import pandas as pd
import numpy as np

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.run_helper

In [2]:
def parse_nullable_int_list(cell):
    try:
        parsed = ast.literal_eval(cell)
        return [int(x) if pd.notna(x) else pd.NA for x in parsed]
    except (ValueError, SyntaxError, TypeError, NameError):
        return [pd.NA]  # or [] if you prefer empty lists for invalid input


def str_list_to_list(str_list: str) -> list[str]:
    ls = str_list[1:-1].split(",")
    ls = [x.strip() if x is not None else x for x in ls]
    return ls


syn_df = pd.read_csv("../data/synthetic.csv")
syn_df["comment_key"] = syn_df.message + syn_df.conv_id

syn_df.Toxicity = syn_df.Toxicity.apply(str_list_to_list).apply(
    lambda x: [int(tox) for tox in x]
)

syn_df.age_annot = syn_df.age_annot.apply(parse_nullable_int_list).apply(
    lambda x : pd.cut(x, bins=4)
)
syn_df.sex_annot = syn_df.sex_annot.apply(str_list_to_list)
syn_df.demographic_group_annot = syn_df.demographic_group_annot.apply(
    str_list_to_list
)
syn_df.sexual_orientation_annot = syn_df.sexual_orientation_annot.apply(
    str_list_to_list
)
syn_df.current_employment_annot = syn_df.current_employment_annot.apply(
    str_list_to_list
)
syn_df.education_level_annot = syn_df.education_level_annot.apply(
    str_list_to_list
)
syn_df

,conv_id,message,intent,Toxicity,Argument Quality,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot,comment_key
0,028f869d-cf12-444a-aefc-62f8b6294acd,@FitnessJunkie07 Respect and honesty are unive...,"['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1]","['25', '25', '25', '25', '25', '25', '25', '25...","['male', 'male', 'male', 'male', 'male', 'male...","['Bisexual', 'Bisexual', 'Bisexual', 'Bisexual...","['Asian', 'Asian', 'Asian', 'Asian', 'Asian', ...","['Software Developer', 'Software Developer', '...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...",@FitnessJunkie07 Respect and honesty are unive...
1,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I agree, but sometimes chang...","['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","['27', '27', '27', '27', '27', '27', '27', '27...","['female', 'female', 'female', 'female', 'fema...","['Lesbian', 'Lesbian', 'Lesbian', 'Lesbian', '...","['Arab', 'Arab', 'Arab', 'Arab', 'Arab', 'Arab...","['Poet', 'Poet', 'Poet', 'Poet', 'Poet', 'Poet...","[""Master's"", ""Master's"", ""Master's"", ""Master's...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I agree, but sometimes chang..."
2,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I completely agree that resp...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I completely agree that resp..."
3,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I understand where you're co...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2]","[2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[(20.956, 32.0], (20.956, 32.0], (20.956, 32.0...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I understand where you're co..."
4,028f869d-cf12-444a-aefc-62f8b6294acd,@LoyalVolunteer54 @FitnessJunkie07 It's import...,"['Moderator', 'Mod

In [3]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="age_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
"(20.956, 32.0]",1.0,1.0,1.0,1.000000,1.0,1.00000,1.0,1.0,1.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,0.780101,1.0,1.000000,1.000000
"(32.0, 43.0]",0.0,1.0,1.0,0.004528,1.0,0.30542,1.0,1.0,0.326452,1.000000,...,0.014019,1.0,0.227059,1.000000,1.000000,1.000000,1.000000,1.0,0.519454,1.000000
"(43.0, 54.0]",0.0,1.0,1.0,1.000000,1.0,1.00000,1.0,1.0,1.000000,0.656917,...,1.000000,1.0,1.000000,1.000000,1.000000,0.120033,1.000000,1.0,0.102230,1.000000
"(54.0, 65.0]",1.0,1.0,1.0,0.831565,1.0,1.00000,1.0,1.0,1.000000,1.000000,...,1.000000,1.0,1.000000,0.095785,0.322079,0.276170,1.000000,1.0,1.000000,0.917439


In [4]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="sexual_orientation_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
'Asexual',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Bisexual',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Heterosexual',1.0,1.0,0.691889,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.248135,1.0,1.0,1.0,1.0,0.115042,1.0,1.0,1.0,1.0
'Homosexual',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Lesbian',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Pansexual',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="education_level_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
"""Bachelor's""",1.0,1.0,1.000000,1.000000,0.088624,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.218089,0.510382,1.000000,1.0,1.000000,1.0,0.686446,1.0
"""Master's""",1.0,1.0,0.076543,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0
'No formal education',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'PhD',1.0,1.0,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.000000,0.065501,0.337999,1.0,0.788321,1.0,1.000000,1.0
'Some College',1.0,1.0,1.000000,0.254275,1.000000,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.000000,1.000000,1.000000,1.0,1.000000,1.0,1.000000,1.0


In [6]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="current_employment_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
'Botanist',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Cybersecurity Expert',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Farmer',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Game Developer',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Historian',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Poet',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Registered Nurse',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Research Scientist',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Retired Philosopher',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Stock Trader',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
tasks.run_helper.results(
    syn_df,
    discussion_id_col="conv_id",
    sdb_column="sex_annot",
    value_col="Toxicity",
    comment_key_col="comment_key",
)

  0%|          | 0/140 [00:00<?, ?it/s]

,028f869d-cf12-444a-aefc-62f8b6294acd,0594f5fe-3bef-4642-8bb7-9ac356f44acb,05d53124-194a-4608-9614-c7075f71a827,07bd0dbc-5591-4b1a-ba75-fb7a84c28986,095f8e2c-4c6b-45d3-983b-c0a836cd1d4e,0d4caeec-0d56-4074-a2df-4bf5012d1c19,11d555f4-f5e4-4f64-a0cd-f89f2b29553d,1257a767-cba5-43c7-a532-deda418022e5,139dce53-526b-4950-83cc-1ce060739e3e,15f0ac31-1302-40eb-ba03-b486561cfd2b,...,ed709c3a-29f6-45f3-a63b-f1f4283d3d89,f4c92f48-1045-49db-9ad5-c5aee0f650cc,f66c6907-aa30-4dc3-8752-fa4b6d926e14,f7221fe4-3eac-42f2-937a-1dacbbe72395,f8085173-8cbf-4c15-a9e8-713c40539bdd,f83f95e7-1200-4dcd-a620-29694858190b,fac8059e-390f-421a-bbb8-a4b615da8d35,fce13b53-ae50-47b2-b70e-1de66f31a8bf,fdb670f7-dd7e-4222-8f5e-e3c6a8fccff4,fe858614-6571-43e7-badf-3cdcb38337b3
'female',0.97817,0.759634,0.751334,0.680985,1.000000,1.000000,1.000000,1.000000,0.613837,0.760098,...,0.109076,1.000000,1.0,1.0,0.676360,1.0,0.147065,0.930118,0.017303,1.0
'male',1.00000,1.000000,1.000000,1.000000,0.509687,0.747853,0.178913,0.197974,0.896663,1.000000,...,1.000000,0.813767,1.0,1.0,0.348742,1.0,1.000000,0.823521,1.000000,1.0
'non-binary',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
